[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# **An Integrated YOLOv8-Based Fire Detection and Automated Alarm Notification System**

---

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog/<BLOG_NAME>)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)
[![Scientific Paper](https://img.shields.io/badge/Official-Paper-blue.svg)](<PAPER LINK>)

## Annotate your Custom dataset using Labellerr

 ***1. Visit the [Labellerr](https://www.labellerr.com/?utm_source=githubY&utm_medium=social&utm_campaign=github_clicks) website and click **“Sign Up”**.*** 

 ***2. After signing in, create your workspace by entering a unique name.***

 ***3. Navigate to your workspace’s API keys page (e.g., `https://<your-workspace>.labellerr.com/workspace/api-keys`) to generate your **API Key** and **API Secret**.***

 ***4. Store the credentials securely, and then use them to initialise the SDK or API client with `api_key`, `api_secret`.*** 


### Use Labellerr SDK for uploading and perform annotation of your own dataset

## 📁 Create & Annotate Your Dataset on Labellerr

Before exporting your annotations, you must set up your project structure on the **Labellerr** platform to ensure your data is organized for the AI model.

1.  **Create a Dataset Folder**: Log in to your Labellerr dashboard and create a new project folder specifically for this fire detection task.
2.  **Upload Raw Data**: Upload your 5G camera frames or video segments into this folder.
3.  **Perform Labeling**: Use the Labellerr interface to draw bounding boxes around fire and smoke regions.
4.  **Export Annotations**: Once labeling is complete, export the data in **COCO JSON format**.

---

## 📥 Download & Import Annotations

Download the generated COCO JSON file from the Labellerr website and upload it into your local project workspace.

This file is the "ground truth" for your AI and will be used for:
* **Frame–Annotation Alignment**: Matching each JSON entry to the correct image frame.
* **Format Conversion**: Converting Labellerr’s COCO format into the **YOLOv8** `.txt` format.
* **Training & Evaluation**: Providing the labeled data required for the model to learn fire patterns.

In [ ]:
# !git clone https://github.com/Labellerr/yolo_finetune_utils.git
from yolo_finetune_utils.coco_yolo_converter.seg_converter import coco_to_yolo_converter

coco_to_yolo_converter(
    json_path="New_Annotation.json",
    images_dir="manufacturing_dataset_frames",
    output_dir="yolo_dataset",
    use_split=True,
    train_ratio=0.7,
    val_ratio=0.2,
    test_ratio=0.1,
    shuffle=True,
    verbose=True
)

# Load and Train YOLO Segmentation Model

Loads the YOLO segmentation model and trains it using the converted YOLO dataset.
- Data: Path to YOLO-style `data.yaml`
- Parameters: epochs, image size, batch size, device, dataloader workers, experiment name.


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.train(
    data="yolo_dataset/data.yaml",
    epochs=25,
    imgsz=640,
    batch=2,
    device="cpu",
    workers=2
)

# Technical Code Overview: Real-Time Fire Detection

This script implements a production-ready fire detection pipeline using **YOLOv8** for object detection and **OpenCV** for real-time stream processing. It is designed to minimize false positives through temporal verification logic before triggering external alerts via a **REST API**.

---

### **1. Key Configuration Parameters**
The script uses several constants to define the sensitivity and behavior of the detection engine:
* **Detection Threshold**: `CONF_THRESHOLD = 0.70` ensures that only detections with high AI confidence are processed.
* **Temporal Filter**: `FRAME_COUNT_THRESHOLD = 3` requires fire to be detected in multiple consecutive frames to trigger an alert.
* **Target Classes**: `FIRE_CLASS_IDS = [0, 1]` specifically monitors for indices representing 'smoke' and 'fire'.
* **Integration Point**: `FASTAPI_SERVER_URL` defines the endpoint where validated alerts are dispatched.

---

### **2. Core Logic & Implementation**

#### **A. Stream Acquisition & Inference**
The system captures video via `cv2.VideoCapture` and utilizes a generator-based inference (`stream=True`) with the **YOLOv8** model. This approach is optimized for memory efficiency during continuous real-time monitoring.

#### **B. Verification Logic (The "Counter" Mechanism)**
To ensure reliability, the script uses a non-persistent counter system:
1. **Detection Check**: For every frame, the script iterates through detected bounding boxes.
2. **Hit**: If a box matches the target class and exceeds the confidence threshold, `found_high_conf_fire` becomes `True`.
3. **Persistence**:
    * If **True**, the `fire_detection_counter` increments.
    * If **False**, the counter immediately resets to `0` (enforcing *consecutive* detection).

#### **C. Automated Alert Dispatch**
Once the counter reaches the threshold (3 frames), the script uses the `requests` library to send an **HTTP POST request** to the backend server. 
* **Error Handling**: The script includes specific exceptions for `ConnectionError` to ensure the main loop doesn't crash if the alert server is offline.
* **Spam Prevention**: The counter is reset immediately after a successful dispatch.

---

### **3. Operational Flowchart**



1. **Input**: Connects to the camera/RTSP stream.
2. **Process**: Runs YOLOv8 inference on each frame.
3. **Logic**: Filters results by class ID and confidence.
4. **Verify**: Tracks consecutive frames to confirm the event.
5. **Action**: Dispatches a POST request and provides visual feedback via `cv2.imshow`.
6. **Cleanup**: Properly releases hardware resources (camera) and closes windows on exit.

In [ ]:
import cv2
import numpy as np
import requests  # <-- ADDED FOR SENDING ALERTS
from ultralytics import YOLO

# --- 1. Configuration ---
RTSP_URL = 0 # 0 for webcam, or your "rtsp://..." string
MODEL_PATH = 'best.pt'
CONF_THRESHOLD = 0.70
FRAME_COUNT_THRESHOLD = 3
FIRE_CLASS_IDS = [0, 1]  # 0='smoke', 1='fire'
FASTAPI_SERVER_URL = "http://127.0.0.1:8000/alert"

# --- 2. Initialization ---
print("Loading model...")
model = YOLO(MODEL_PATH)

print(f"Connecting to video stream ({RTSP_URL})...")
try:
    cap = cv2.VideoCapture(RTSP_URL)
    if not cap.isOpened():
        raise Exception("Cannot open video capture.")
    print("Video stream connected successfully.")
except Exception as e:
    print(f"--- ERROR: Failed to connect to video stream. ---")
    print(f"--- If using a webcam, is it in use by another app? ---")
    print(f"--- Error details: {e} ---")
    exit()

fire_detection_counter = 0
print("--- Starting fire detection loop (Press 'q' to quit) ---")

# --- 3. Main Processing Loop ---
while True:
    ret, frame = cap.read()
    if not ret:
        print("Stream ended or error.")
        break

    # --- 4. AI Model Inference ---
    results = model(frame, stream=True, verbose=False)

    found_high_conf_fire = False

    # --- 5. Process Results (Apply Logic) ---
    for result in results:
        boxes = result.boxes
        for box in boxes:
            confidence = box.conf[0]
            class_id = int(box.cls[0])

            if class_id in FIRE_CLASS_IDS and confidence >= CONF_THRESHOLD:
                found_high_conf_fire = True

                # --- (Optional) Draw a box on the frame ---
                # --- NEW, CORRECT LINE ---
                x1, y1, x2, y2 = np.array(box.xyxy[0], dtype=int)
                label = f"{model.names[class_id]}: {confidence:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                break  # Only need one detection per frame

    # --- 6. Implement "3+ Frames" Logic ---
    if found_high_conf_fire:
        fire_detection_counter += 1
    else:
        fire_detection_counter = 0  # Reset counter if no fire is found

    # --- 7. Trigger Alert ---
    if fire_detection_counter >= FRAME_COUNT_THRESHOLD:
        print(f"!!! ALERT: Fire detected for {fire_detection_counter} frames! Sending to server... !!!")
        try:
            # Send an HTTP POST request to your FastAPI server
            r = requests.post(FASTAPI_SERVER_URL)
            r.raise_for_status()  # Raise an exception if the server returns an error
            print("--- Alert successfully sent to server. ---")
        except requests.exceptions.ConnectionError:
            print("--- ERROR: Could not connect to FastAPI server. Is it running? ---")
        except Exception as e:
            print(f"--- ERROR: An unknown error occurred: {e} ---")

        # Reset counter after sending alert to avoid spamming
        fire_detection_counter = 0

    # --- (Optional) Display the video feed ---
    cv2.imshow("Fire Detection Stream", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- 8. Cleanup ---
cap.release()
cv2.destroyAllWindows()
print("--- Detection stopped. ---")

# Alert Server Overview: Multi-Channel Emergency Response

This backend script acts as the **Alert Orchestration Layer** of the fire detection ecosystem. It transforms a technical detection signal into an actionable emergency response by simultaneously triggering three communication channels: **Email**, **SMS**, and **Voice Call**.

---

### **1. Key Functional Components**

* **FastAPI Endpoint (`/alert`)**: A high-performance REST API that listens for POST requests from the detection script. When triggered, it executes the notification cascade in parallel.
* **Email Alerting (`smtplib`)**: Connects to Gmail’s SMTP server using **SSL/TLS** to send a high-priority alert to the specified administrator email.
* **Twilio SMS Integration**: Utilizes the Twilio REST API to send an instantaneous text message alert to a verified mobile device.
* **Twilio Voice Call (`make_voice_call`)**: Initiates an automated phone call that speaks the custom alert message defined in your **TwiML Bin**.
    * **Auth Persistence**: The script specifically appends the `AccountSid` to the TwiML URL to ensure seamless authentication during the call request.

---

### **2. Technical Stack Mapping**

| Feature | Technology | Role in Architecture |
| :--- | :--- | :--- |
| **API Framework** | **FastAPI** | Receives signals from the YOLOv8/NumPy logic stage. |
| **Server** | **Uvicorn** | Manages the high-speed ASGI server environment. |
| **Email Protocol** | **SMTP** | Handles secure transmission of emergency emails. |
| **SMS/Voice** | **Twilio Client** | Manages external telephony and messaging gateways. |
| **Security** | **python-dotenv** | Manages sensitive API keys and credentials securely. |

---

### **3. Operational Workflow**

1.  **System Check**: On startup, the script validates that all five required environment variables (Gmail and Twilio secrets) are correctly loaded.
2.  **Listening Mode**: The server waits at `http://127.0.0.1:8000/alert` for incoming signals from the 5G AI Camera pipeline.
3.  **Trigger Activation**: Once the **YOLOv8 logic** confirms fire persistence, it sends a POST request to this server.
4.  **Multi-Channel Dispatch**:
    * **Email** is sent via Gmail's secure tunnel.
    * **SMS** is dispatched via Twilio's messaging service.
    * **Voice Call** is triggered using the TwiML Bin URL for audio instructions.
5.  **Confirmation**: The server logs the SIDs of the sent alerts and returns a success status to the detection script.

In [ ]:
from fastapi import FastAPI
import uvicorn
import smtplib
import ssl
import os
from twilio.rest import Client
from dotenv import load_dotenv

load_dotenv()
# --- Email Configuration ---
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587
SENDER_EMAIL = "aaryanaggarwal2005@gmail.com"
RECEIVER_EMAIL = "aaryanaggarwal.bt23ece@pec.edu.in"

# --- Twilio Configuration ---
TO_PHONE_NUMBER = "+919915840484" # Phone number to alert

# --- Load all secrets from environment variables ---
EMAIL_PASSWORD = os.environ.get("GMAIL_APP_PASSWORD")
TWILIO_ACCOUNT_SID = os.environ.get("TWILIO_ACCOUNT_SID")
TWILIO_AUTH_TOKEN = os.environ.get("TWILIO_AUTH_TOKEN")
TWILIO_PHONE_NUMBER = os.environ.get("TWILIO_PHONE_NUMBER")
TWIML_BIN_URL = os.environ.get("TWIML_BIN_URL")  # <-- 1. ADD NEW VARIABLE
# --------------------------------------------------

app = FastAPI()

# (Your send_email() function stays here, unchanged)
def send_email():
    if not EMAIL_PASSWORD:
        print("--- ERROR: Cannot send email. Password is not set. ---")
        return
    # ... (rest of your email code is here) ...
    message = """\
Subject: !!! FIRE ALERT DETECTED !!!

A fire or smoke event has been confirmed by the AI detection system.
Please check the cameras immediately.
"""
    context = ssl.create_default_context()
    print("--- Connecting to Gmail server to send alert... ---")
    try:
        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls(context=context)
        server.login(SENDER_EMAIL, EMAIL_PASSWORD)
        server.sendmail(SENDER_EMAIL, RECEIVER_EMAIL, message)
        server.quit()
        print("--- Email alert successfully sent! ---")
    except smtplib.SMTPException as e:
        print(f"--- ERROR: Failed to send email: {e} ---")
    except Exception as e:
        print(f"--- An unexpected error occurred: {e} ---")


# (Your send_sms() function stays here, unchanged)
def send_sms():
    if not all([TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, TWILIO_PHONE_NUMBER]):
        print("--- ERROR: Twilio credentials not set. Cannot send SMS. ---")
        return
    # ... (rest of your SMS code is here) ...
    print("--- Connecting to Twilio to send SMS... ---")
    try:
        client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
        message = client.messages.create(
            body="!!! FIRE ALERT DETECTED !!! Check cameras immediately.",
            from_=TWILIO_PHONE_NUMBER,
            to=TO_PHONE_NUMBER
        )
        print(f"--- SMS alert successfully sent! (SID: {message.sid}) ---")
    except Exception as e:
        print(f"--- ERROR: Failed to send SMS: {e} ---")


# --- 2. ADD NEW MAKE_VOICE_CALL FUNCTION ---
def make_voice_call():
    """
    Uses Twilio to make an automated voice call.
    """
    if not all([TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, TWILIO_PHONE_NUMBER, TWIML_BIN_URL]):
        print("--- ERROR: Twilio credentials or TwiML URL not set. Cannot make call. ---")
        return

    print("--- Connecting to Twilio to make voice call... ---")
    try:
        client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
        
        # Add AccountSid as query parameter to TwiML Bin URL
        twiml_url_with_auth = f"{TWIML_BIN_URL}?AccountSid={TWILIO_ACCOUNT_SID}"
        
        call = client.calls.create(
            url=twiml_url_with_auth,  # <-- Tells Twilio what to say
            from_=TWILIO_PHONE_NUMBER,
            to=TO_PHONE_NUMBER
        )
        print(f"--- Voice call initiated! (SID: {call.sid}) ---")
    except Exception as e:
        print(f"--- ERROR: Failed to make voice call: {e} ---")
# ---------------------------------------------


@app.post("/alert")
async def receive_alert():
    """
    This endpoint triggers email, SMS, and a voice call.
    """
    print("-----------------------------------------")
    print("!!! ALERT RECEIVED FROM DETECTION SCRIPT !!!")
    print("--- Fire or smoke has been confirmed. ---")
    print("-----------------------------------------")
    
    # --- 3. CALL ALL THREE FUNCTIONS ---
    send_email()
    send_sms()
    make_voice_call()
    # ---------------------------------
    
    return {"status": "alert received, notifications triggered"}


if __name__ == "__main__":
    if not all([EMAIL_PASSWORD, TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, TWILIO_PHONE_NUMBER, TWIML_BIN_URL]):
        print("--- WARNING: One or more environment variables are missing! ---")
        print("--- Please set all 5 variables: GMAIL, TWILIO_SID, TWILIO_TOKEN, TWILIO_PHONE, and TWIML_BIN_URL ---")

    print("Starting FastAPI server on http://127.0.0.1:8000")
    uvicorn.run(app, host="127.0.0.1", port=8000)

---

## 👨‍💻 About Labellerr's Hands-On Learning in Computer Vision

Thank you for exploring this **Labellerr Hands-On Computer Vision Cookbook**! We hope this notebook helped you learn, prototype, and accelerate your vision projects.  
Labellerr provides ready-to-run Jupyter/Colab notebooks for the latest models and real-world use cases in computer vision, AI agents, and data annotation.

---
## 🧑‍🔬 Check Our Popular Youtube Videos

Whether you're a beginner or a practitioner, our hands-on training videos are perfect for learning custom model building, computer vision techniques, and applied AI:

- [How to Fine-Tune YOLO on Custom Dataset](https://www.youtube.com/watch?v=pBLWOe01QXU)  
  Step-by-step guide to fine-tuning YOLO for real-world use—environment setup, annotation, training, validation, and inference.
- [Build a Real-Time Intrusion Detection System with YOLO](https://www.youtube.com/watch?v=kwQeokYDVcE)  
  Create an AI-powered system to detect intruders in real time using YOLO and computer vision.
- [Finding Athlete Speed Using YOLO](https://www.youtube.com/watch?v=txW0CQe_pw0)  
  Estimate real-time speed of athletes for sports analytics.
- [Object Counting Using AI](https://www.youtube.com/watch?v=smsjBBQcIUQ)  
  Learn dataset curation, annotation, and training for robust object counting AI applications.
---

## 🎦 Popular Labellerr YouTube Videos

Level up your skills and see video walkthroughs of these tools and notebooks on the  
[Labellerr YouTube Channel](https://www.youtube.com/@Labellerr/videos):

- [How I Fixed My Biggest Annotation Nightmare with Labellerr](https://www.youtube.com/watch?v=hlcFdiuz_HI) – Solving complex annotation for ML engineers.
- [Explore Your Dataset with Labellerr's AI](https://www.youtube.com/watch?v=LdbRXYWVyN0) – Auto-tagging, object counting, image descriptions, and dataset exploration.
- [Boost AI Image Annotation 10X with Labellerr's CLIP Mode](https://www.youtube.com/watch?v=pY_o4EvYMz8) – Refine annotations with precision using CLIP mode.
- [Boost Data Annotation Accuracy and Efficiency with Active Learning](https://www.youtube.com/watch?v=lAYu-ewIhTE) – Speed up your annotation workflow using Active Learning.

> 👉 **Subscribe** for Labellerr's deep learning, annotation, and AI tutorials, or watch videos directly alongside notebooks!

---

## 🤝 Stay Connected

- **Website:** [https://www.labellerr.com/](https://www.labellerr.com/)
- **Blog:** [https://www.labellerr.com/blog/](https://www.labellerr.com/blog/)
- **GitHub:** [Labellerr/Hands-On-Learning-in-Computer-Vision](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)
- **LinkedIn:** [Labellerr](https://in.linkedin.com/company/labellerr)
- **Twitter/X:** [@Labellerr1](https://x.com/Labellerr1)

*Happy learning and building with Labellerr!*
